# Load the Population Receptive Field properties for the Connective Fields

In [20]:
import pandas as pd
import numpy as np
import os 
import pickle 
from nibabel.freesurfer.io import read_morph_data, write_morph_data
import io

In [21]:
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
freesurfer = f"{MAIN_PATH}/freesurfer"
source = 'V1'
atlas = 'manual'
delineation = 'benson'
denoising = 'nordic'

subjects = [f'sub-{i:02}' for i in range(46, 46)]
# tasks = ['RET2','RET', 'RestingState']
tasks = ['RET2','RET']
hemispheres = ['lh', 'rh']
visual_areas = ['V1', 'V2', 'V3', 'LO', 'V4']

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from nibabel.freesurfer import read_morph_data

class PRFModel:
    def __init__(self, r2, size, ecc, angle):
        self.r2 = r2
        self.size = size
        self.ecc = ecc
        self.angle = angle

def load_prf(subj, main_path, atlas, denoising, task):
    if task == 'RestingState':
        task_for_file = 'RET'
    else:
        task_for_file = task

    for ses in ['ses-01', 'ses-02']:
        filepath = os.path.join(
            main_path,
            f'pRFM/{subj}/{ses}/{denoising}/model-{atlas}-nelder-mead-GM_desc-prf_params_{task_for_file}.pkl'
        )
        if os.path.exists(filepath):
            with open(filepath, 'rb') as file:
                pkl_data = pickle.load(file)
            prf_params = pkl_data['model'].iterative_search_params
            prf_voxels = np.where(pkl_data['rois_mask'] == 1)[0]
            return prf_params, prf_voxels

    raise FileNotFoundError(f"pRF .pkl file not found for subject {subj}.")

def filter_prf(prf_params):
    return PRFModel(
        r2=prf_params[:, 7],
        size=prf_params[:, 2],
        ecc=np.sqrt(prf_params[:, 1]**2 + prf_params[:, 0]**2),
        angle=np.arctan2(prf_params[:, 1], prf_params[:, 0])
    )

def get_benson_vertices(label_file):
    df = pd.read_csv(label_file, sep='\s+', skiprows=2, header=None)
    df.columns = ['vertex', 'x', 'y', 'z', 'value']
    return df['vertex'].values

def source_eccentricity_benson(subj, hemi, main_path, atlas, denoising, task, freesurfer_path, label_file):
    prf_params, prf_voxels = load_prf(subj, main_path, atlas, denoising, task)
    prf_model = filter_prf(prf_params)

    lh_c = read_morph_data(os.path.join(freesurfer_path, subj, 'surf', 'lh.curv'))
    numel_lh = lh_c.shape[0]

    if hemi == 'rh':
        adjusted_voxels = prf_voxels[prf_voxels >= numel_lh] - numel_lh
        ecc = prf_model.ecc[prf_voxels >= numel_lh]
    else:
        adjusted_voxels = prf_voxels[prf_voxels < numel_lh]
        ecc = prf_model.ecc[prf_voxels < numel_lh]

    ecc_dict = dict(zip(adjusted_voxels, ecc))

    # Load Benson vertices
    benson_vertices = get_benson_vertices(label_file)

    # Filter vertices by eccentricity range
    filtered_vertices = [v for v in benson_vertices if v in ecc_dict and 0.5 <= ecc_dict[v] <= 10]

    print(f"Total Benson vertices: {len(benson_vertices)}")
    print(f"Filtered vertices (0.5 <= ecc <= 10): {len(filtered_vertices)}")
    print(f"Example: {[(v, round(ecc_dict[v], 2)) for v in filtered_vertices[:5]]}")

    return filtered_vertices

# ==== USAGE ====
subj = 'sub-46'
hemi = 'lh'
main_path = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives'
atlas = 'manual'
denoising = 'nordic'
task = 'RET'
freesurfer_path = f"{main_path}/freesurfer"
label_file = f"{freesurfer_path}/{subj}/label/{hemi}.benson14_varea-0001.label"

filtered_vertices = source_eccentricity_benson(subj, hemi, main_path, atlas, denoising, task, freesurfer_path, label_file)

Total Benson vertices: 125206
Filtered vertices (0.5 <= ecc <= 10): 7383
Example: [(0, 4.24), (1, 4.16), (2, 4.23), (3, 4.26), (4, 4.21)]


In [22]:
class PRFModel:
    def __init__(self, r2, size, ecc, angle):
        self.r2 = r2        
        self.size = size
        self.ecc = ecc    
        self.angle = angle     

def pickle_file(filepath):
    if not os.path.isfile(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    with open(filepath, 'rb') as file: 
        return pickle.load(file)

def load_prf(subj, main_path, atlas, denoising, task):
    filename_task = 'RET' if task == 'RestingState' else task
    filepath = os.path.join(
        main_path,
        f'pRFM/{subj}/ses-02/{denoising}/model-{atlas}-nelder-mead-GM_desc-prf_params_{filename_task}.pkl')
    pkl_data = pickle_file(filepath)
    prf_params = pkl_data['model'].iterative_search_params     
    prf_voxels = np.where(pkl_data['rois_mask'] == 1)[0]
    return prf_params, prf_voxels

def filter_prf(prf_params, prf_voxels):
    return PRFModel(
        r2=prf_params[:, 7],
        size=prf_params[:, 2],
        ecc=np.sqrt(prf_params[:, 1]**2 + prf_params[:, 0]**2),
        angle=np.arctan2(prf_params[:, 1], prf_params[:, 0]))

def extract_prf(subj, hemi, target, csv_path, prf_params, prf_voxels):
    best_fit = pd.read_csv(csv_path)

    if 'Target Vertex Index' in best_fit.columns:
        repeated_header_rows = best_fit['Target Vertex Index'] == 'Target Vertex Index'
        if repeated_header_rows.any():
            best_fit = best_fit[~repeated_header_rows]
            # print(f"Removed repeated header row in {csv_path}")

    best_fit['Source Vertex Index'] = best_fit['Source Vertex Index'].astype(int)
    best_fit['Target Vertex Index'] = best_fit['Target Vertex Index'].astype(int)

    prf_model = filter_prf(prf_params, prf_voxels)
    lh_c = read_morph_data(f'{freesurfer}/{subj}/surf/lh.curv')
    numel_lh = lh_c.shape[0]

    if hemi == 'rh':
        prf_voxels_rh = prf_voxels[prf_voxels >= numel_lh]
        prf_voxels_rh_adjusted = prf_voxels_rh - numel_lh
        prf_ecc_rh = prf_model.ecc[prf_voxels >= numel_lh]
        prf_angle_rh = prf_model.angle[prf_voxels >= numel_lh]
        voxel_to_ecc = dict(zip(prf_voxels_rh_adjusted, prf_ecc_rh))
        voxel_to_angle = dict(zip(prf_voxels_rh_adjusted, prf_angle_rh))
    else:
        voxel_to_ecc = dict(zip(prf_voxels, prf_model.ecc))
        voxel_to_angle = dict(zip(prf_voxels, prf_model.angle))

    best_fit['Source Eccentricity'] = best_fit['Source Vertex Index'].map(voxel_to_ecc)
    best_fit['Source Polar Angle'] = best_fit['Source Vertex Index'].map(voxel_to_angle)
    return best_fit

def header_fix(path, expected_column='Target Vertex Index'):
    with open(path, 'r') as f:
        lines = f.readlines()
    for idx, line in enumerate(lines):
        if expected_column in line:
            header = line
            data = lines[idx + 1:]
            break
    csv_content = ''.join([header] + data)
    return pd.read_csv(io.StringIO(csv_content))


In [23]:
for subj in subjects:
    for task in tasks:
        try:
            prf_params, prf_voxels = load_prf(subj, MAIN_PATH, atlas, denoising, task)
        except Exception as e:
            print(f"Missing pRF pkl file of Subject: {subj}, Task: {task}: {e}")
            continue

        for hemi in hemispheres:
            for target in visual_areas:
                #best_fit_path = f'{MAIN_PATH}/CFM/{subj}/ses-1/GM/{hemi}/{denoising}/{task}/{target}/best_fits.csv'
                #best_fit_path = f'{MAIN_PATH}/CFM/{subj}/ses-02/{atlas}/{task}/{denoising}/GM/{target}/{hemi}/{source}-{target}/best_fits.csv'
                best_fit_path = f'{{MAIN_PATH}}/CFM/{subj}/ses-02/{atlas}/{task}/{denoising}/1/GM/lh/V1-V1/best_fits.csv'
                if not os.path.exists(best_fit_path):
                    print(f'Missing File of {subj}, {target}, {task}, {hemi}')
                    continue

                try:
                    best_fit_prf = extract_prf(subj, hemi, target, best_fit_path, prf_params, prf_voxels)
                except Exception as e:
                    if isinstance(e, KeyError) and 'Source Vertex Index' in str(e):
                        best_fit = header_fix(best_fit_path)
                        best_fit.to_csv(best_fit_path, index=False) 
                        best_fit_prf = extract_prf(subj, hemi, target, best_fit_path, prf_params, prf_voxels)

                output_path = f'{MAIN_PATH}/CFM/{subj}/ses-1/GM/{hemi}/{denoising}/{task}/{target}/best_fits_prf.csv'
                best_fit_prf.to_csv(output_path, index=False)

                target_idx = best_fit_prf['Target Vertex Index'].astype(int).values
                curv = read_morph_data(f'{freesurfer}/{subj}/surf/{hemi}.curv')

                def write_data_to_surface(values, suffix):
                    masked = np.full(curv.shape[0], 50.0)
                    masked[target_idx] = values
                    output_file = f'{freesurfer}/{subj}/surf/{hemi}.{suffix}_{target}{source}_{denoising}_{task}'
                    write_morph_data(output_file, masked)

                write_data_to_surface(best_fit_prf['Source Eccentricity'].values, 'ecc')
                write_data_to_surface(best_fit_prf['Source Polar Angle'].values, 'pol')
                write_data_to_surface(best_fit_prf['Best Variance Explained Finer'].values, 've')
                write_data_to_surface(best_fit_prf['Best Sigma Finer'].values, 'sigma')